# Merge the baroclinic/barotropic results files to make more usable

In [18]:
import xarray as xr
# import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

In [52]:
start = dt.datetime(2019,2,28)
numdays = np.linspace(0,15*4,5)
date_list = [start + dt.timedelta(days=x) for x in numdays]
date_list[-1] = date_list[-1] + dt.timedelta(days=1) #because the last run of the month is a day longer

In [53]:
date_list

[datetime.datetime(2019, 2, 28, 0, 0),
 datetime.datetime(2019, 3, 15, 0, 0),
 datetime.datetime(2019, 3, 30, 0, 0),
 datetime.datetime(2019, 4, 14, 0, 0),
 datetime.datetime(2019, 4, 30, 0, 0)]

In [54]:
files = ["/data/rbeutel/analysis/ssc_tidesback/u_new_{:%d%b%y}_{:%d%b%y}.nc".format(date_list[i],date_list[i+1])for i in range(len(date_list)-1)]

In [56]:
mydata = xr.open_mfdataset(files)
mydata

<xarray.Dataset>
Dimensions:                        (time_counter: 1468, y: 898, x: 398, depthu: 40)
Coordinates:
  * time_counter                   (time_counter) datetime64[ns] 2019-02-28T1...
    nav_lat                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 361), meta=np.ndarray>
    nav_lon                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 361), meta=np.ndarray>
  * depthu                         (depthu) float32 0.5 1.5 2.5 ... 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    __xarray_dataarray_variable__  (time_counter, y, x, depthu) float32 dask.array<chunksize=(361, 898, 398, 40), meta=np.ndarray>

In [88]:
#we dont want the extra half day on either end, so remove the first and last 12 hours
inds = np.arange(12,len(mydata.time_counter)-13)
#we also get repeats of 1 hour between each merge, remove the second of these two
inds = np.delete(inds, [361,361*2,361*3])

In [89]:
# trim the times we don't need
trimdata = mydata.isel(time_counter = inds)
trimdata

<xarray.Dataset>
Dimensions:                        (time_counter: 1440, y: 898, x: 398, depthu: 40)
Coordinates:
  * time_counter                   (time_counter) datetime64[ns] 2019-03-01T0...
    nav_lat                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 349), meta=np.ndarray>
    nav_lon                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 349), meta=np.ndarray>
  * depthu                         (depthu) float32 0.5 1.5 2.5 ... 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    __xarray_dataarray_variable__  (time_counter, y, x, depthu) float32 dask.array<chunksize=(349, 898, 398, 40), meta=np.ndarray>

In [91]:
path = '/data/rbeutel/analysis/ssc_tidesback/'
trimdata.to_netcdf(str(path)+'u_new_MarchRun')

In [92]:
files = ["/data/rbeutel/analysis/ssc_tidesback/v_new_{:%d%b%y}_{:%d%b%y}.nc".format(date_list[i],date_list[i+1])for i in range(len(date_list)-1)]
mydata = xr.open_mfdataset(files)

In [93]:
# trim the times we don't need
trimdata = mydata.isel(time_counter = inds)
trimdata

/home/rbeutel/anaconda3/envs/py39/lib/python3.9/site-packages/xarray/core/indexing.py:1226: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


<xarray.Dataset>
Dimensions:                        (time_counter: 1440, y: 898, x: 398, depthv: 40)
Coordinates:
  * time_counter                   (time_counter) datetime64[ns] 2019-03-01T0...
    nav_lat                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 349), meta=np.ndarray>
    nav_lon                        (y, x, time_counter) float32 dask.array<chunksize=(898, 398, 349), meta=np.ndarray>
  * depthv                         (depthv) float32 0.5 1.5 2.5 ... 414.5 441.5
Dimensions without coordinates: y, x
Data variables:
    __xarray_dataarray_variable__  (time_counter, y, x, depthv) float32 dask.array<chunksize=(349, 898, 398, 40), meta=np.ndarray>

In [94]:
path = '/data/rbeutel/analysis/ssc_tidesback/'
trimdata.to_netcdf(str(path)+'v_new_MarchRun')